<a href="https://colab.research.google.com/github/ariqnfl/IYKRA-DF6/blob/main/PySpark_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA CLEANING (TECHNICAL CODING ROUND)
__ Creator : Rahul Muliana - SDE __

### __IMPORT THE DATASET__ 

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar -xvzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

In [ ]:
#Add environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


# findspark will locate spark in the system

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import format_number
from pyspark.sql.functions import max, min
from pyspark.sql.functions import mean
from pyspark.sql.functions import corr
from pyspark.sql.functions import year
from pyspark.sql.functions import month

In [ ]:
# set spark session

spark = SparkSession.builder \
        .master("local") \
        .appName("spark_learn") \
        .getOrCreate()

In [ ]:
df = spark.read.format("csv").options(header='true', inferSchema='true').load('/content/BL-Flickr-Images-Book.csv')

In [ ]:
df.printSchema()

root
 |-- Identifier: integer (nullable = true)
 |-- Edition Statement: string (nullable = true)
 |-- Place of Publication: string (nullable = true)
 |-- Date of Publication: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Contributors: string (nullable = true)
 |-- Corporate Author: string (nullable = true)
 |-- Corporate Contributors: string (nullable = true)
 |-- Former owner: string (nullable = true)
 |-- Engraver: string (nullable = true)
 |-- Issuance type: string (nullable = true)
 |-- Flickr URL: string (nullable = true)
 |-- Shelfmarks: string (nullable = true)



#### __READING THE FIRST FEW ENTRIES OF THE DATASET__

In [ ]:
# BL-Flickr-Images-Book.csv
df.show(5)

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|Identifier|   Edition Statement|Place of Publication|Date of Publication|           Publisher|               Title|   Author|        Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|          Flickr URL|          Shelfmarks|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|       206|                null|              London|        1879 [1878]|    S. Tinsley & Co.|Walter Forbes. [A...|    A. A.|     FORBES, Walter.|            null|                  null|        null|  

## Cleaning the messy columns

 __Some data entries are really messy and needs to be cleaned as they contain several values which makes it hard to analyse the data __

__ The Date of Publication column is slightly messy. It would be better if we only had years in the column. Let us see how you can clean it __

In [ ]:
import numpy as np

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [ ]:
import re

In [ ]:
# make DEF Function

def clean_dates(item):
    dop= str(item)
    dop = re.findall(r'\d{4}',dop)
    if len(dop) >= 1:
      dop = dop[0]
    elif len(dop) <1:
      dop = 1800
    dop = int(dop)
    return dop

In [ ]:
clean_dates('2021')

2021

In [ ]:
transform_dates_udf = F.udf(f=lambda x : clean_dates(x))

In [ ]:
df = df.withColumn("Date of Publication", transform_dates_udf(F.col("Date of Publication")))
df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|Identifier|   Edition Statement|Place of Publication|Date of Publication|           Publisher|               Title|              Author|        Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|          Flickr URL|          Shelfmarks|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|       206|                null|              London|               1879|    S. Tinsley & Co.|Walter Forbes. [A...|               A. A.|     FORBES, Walter.|           

__ The next such column is the 'Title' column. Let us try to clean that column now. __

In [ ]:
# make DEF Function
def clean_title(title):
    if title == 'nan':
        return 'NaN'
    
    if title[0] == '[':
        title = title[1: title.find(']')]
        
    if 'by' in title:
        title = title[:title.find('by')]
    elif 'By' in title:
        title = title[:title.find('By')]
        
    if '[' in title:
        title = title[:title.find('[')]

    title = title[:-2]
        
    title = list(map(str.capitalize, title.split()))
    return ' '.join(title)

In [ ]:
transform_title_udf = F.udf(f=lambda x : clean_title(x), returnType=T.StringType())

In [ ]:
df =df.withColumn("Title", transform_title_udf(F.col("Title")))
df.show(5)

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|Identifier|   Edition Statement|Place of Publication|Date of Publication|           Publisher|               Title|   Author|        Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|          Flickr URL|          Shelfmarks|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+---------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|       206|                null|              London|               1879|    S. Tinsley & Co.|       Walter Forbes|    A. A.|     FORBES, Walter.|            null|                  null|        null|  

__ The 'Author' column needs to be tweaked a little too __

In [ ]:
from functools import reduce

In [ ]:
def clean_author_names(author):
    
    author = str(author)
    
    if author == 'nan':
        return 'NaN'
    
    author = author.split(',')

    if len(author) == 1:
        name = filter(lambda x: x.isalpha(), author[0])
        return reduce(lambda x, y: x + y, name)
    
    last_name, first_name = author[0], author[1]

    first_name = first_name[:first_name.find('-')] if '-' in first_name else first_name
    
    if first_name.endswith(('.', '.|')):
        parts = first_name.split('.')
        
        if len(parts) > 1:
            first_occurence = first_name.find('.')
            final_occurence = first_name.find('.', first_occurence + 1)
            first_name = first_name[:final_occurence]
        else:
            first_name = first_name[:first_name.find('.')]
    
    last_name = last_name.capitalize()
    
    return f'{first_name} {last_name}'

In [ ]:
clean_author_names('A. A.')

'AA'

In [ ]:
transform_author_udf = F.udf(f=lambda x : clean_author_names(x))

In [ ]:
df = df.withColumn("Author", transform_author_udf(F.col("Author")))
df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|Identifier|   Edition Statement|Place of Publication|Date of Publication|           Publisher|               Title|              Author|        Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|          Flickr URL|          Shelfmarks|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|       206|                null|              London|               1879|    S. Tinsley & Co.|       Walter Forbes|                  AA|     FORBES, Walter.|           

__ The 'Place of publication' has a few complications too. Especially with places containing London and Oxford. __

In [ ]:
!pip install https://github.com/elyase/geotext/archive/master.zip

     - 2.3 MB 317 kB/s
  Created wheel for geotext: filename=geotext-0.3.0-py2.py3-none-any.whl size=2272483 sha256=f18ca66ce62fad0a1056e3093d52cd6ca7ae62c7c6a398aad4ab36257d9b27ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-f_e289ol/wheels/c3/82/e9/208ca98c5e7fd6edcd87669e8270db1d858cd078d230452cd5
Successfully built geotext


In [ ]:
from string import punctuation
from geotext import GeoText

In [ ]:
# make DEF Function
def clean_place(place):
    place = str(place)
    place = GeoText(place)
    place = place.cities
    if len(place) > 1:
      return ''.join(place[-1])
    elif len(place) == 1:
      return ' '.join(place)
    else:
      return 'Nan'
    return place

In [ ]:
clean_place('sadfsa')

'Nan'

In [ ]:
transform_place_udf = F.udf(lambda x : clean_place(x))

In [ ]:
df = df.withColumn("Place of Publication", transform_place_udf(F.col("Place of Publication")))

In [ ]:
df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|Identifier|   Edition Statement|Place of Publication|Date of Publication|           Publisher|               Title|              Author|        Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|          Flickr URL|          Shelfmarks|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|       206|                null|              London|               1879|    S. Tinsley & Co.|       Walter Forbes|                  AA|     FORBES, Walter.|           

In [ ]:
df.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|Identifier|   Edition Statement|Place of Publication|Date of Publication|           Publisher|               Title|              Author|        Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|          Flickr URL|          Shelfmarks|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------------+----------------------+------------+--------+-------------+--------------------+--------------------+
|       206|                null|              London|               1879|    S. Tinsley & Co.|Walter Forbes. [A...|               A. A.|     FORBES, Walter.|           

### Detecting missing values and dealing with them
__The missing values could be in many ways. Most common ways of missing values are the NaN values or nothing at all being present in the entries. Let us first look at what to do with the NaN / NULL / Blank Character values__

__Let us first count the number of values which are NaN / NULL / Blank Character and etc__

In [ ]:
df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+----------+-----------------+--------------------+-------------------+---------+-----+------+------------+----------------+----------------------+------------+--------+-------------+----------+----------+
|Identifier|Edition Statement|Place of Publication|Date of Publication|Publisher|Title|Author|Contributors|Corporate Author|Corporate Contributors|Former owner|Engraver|Issuance type|Flickr URL|Shelfmarks|
+----------+-----------------+--------------------+-------------------+---------+-----+------+------------+----------------+----------------------+------------+--------+-------------+----------+----------+
|         0|             7514|                   0|                  0|     4195|    0|     0|           2|            8276|                  8282|        8282|    8284|           10|         5|         4|
+----------+-----------------+--------------------+-------------------+---------+-----+------+------------+----------------+----------------------+------------+--------+-------

In [ ]:
df.describe().show()

+-------+------------------+--------------------+--------------------+-------------------+-------------------+--------------+--------------------+--------------------+------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|        Identifier|   Edition Statement|Place of Publication|Date of Publication|          Publisher|         Title|              Author|        Contributors|  Corporate Author|Corporate Contributors|        Former owner|            Engraver|       Issuance type|          Flickr URL|          Shelfmarks|
+-------+------------------+--------------------+--------------------+-------------------+-------------------+--------------+--------------------+--------------------+------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  count|              8287|                 773|      